In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
pd.options.mode.chained_assignment = None  # default='warn'

# Exploración de Datos

In [ ]:
dfToUse = "consumo"

def returnQuantRows(dfToUse):
    if (dfToUse == 'consumo'):
        return 269 
    return 257

toUse = returnQuantRows(dfToUse)

In [ ]:
df = pd.read_excel(dfToUse+'.xlsx', engine='openpyxl')
df = df[['Fecha', 'Gasolina superior', 'Gasolina regular', 'Diesel']]

In [ ]:
df = df[:toUse]
df['Fecha'] = pd.to_datetime(df['Fecha'])

In [ ]:
df

In [ ]:
quant_vars = ['Gasolina superior', 'Gasolina regular', 'Diesel']
df[quant_vars].astype(float).describe()

In [ ]:
for var in quant_vars:
    data = df[var].dropna(how='all', axis=0)
    
    # Gráfico
    sns.displot(data, kde=True)
    # print('\033[1m' + var + '\033[0m' + ': Kurtosis:', stats.kurtosis(data), 'Skewness:', stats.skew(data), '\n')


# Por año

In [ ]:
plt.rcParams["figure.figsize"] = (20,5.5)

for gas in quant_vars:
    x = df['Fecha']
    y = df[gas]

    plt.title(dfToUse+" 2000-2022")
    # beautify the x-labels
    plt.gcf().autofmt_xdate()
    plt.xlabel(gas)


    plt.plot(x, y)
    plt.show()

# Por mes

In [ ]:
dfPerMonth = df.groupby(df['Fecha'].dt.month)
dfPerMonth = dfPerMonth.sum()

In [ ]:
plt.rcParams["figure.figsize"] = (20,5.5)

for gas in quant_vars:
    x = dfPerMonth.index
    y = dfPerMonth[gas]


    plt.title(dfToUse+" por mes")
    # beautify the x-labels
    plt.gcf().autofmt_xdate()
    plt.xlabel(gas)


    plt.bar(x, y)
    plt.show()

# Comportamiento en la pandemia

In [ ]:
plt.rcParams["figure.figsize"] = (20,5.5)
quant_to_have = 12

for gas in quant_vars:
    x = df['Fecha'].tail(quant_to_have * 3)
    y = df[gas].tail(quant_to_have * 3)

    plt.title(dfToUse+" durante pandemia")
    # beautify the x-labels
    plt.gcf().autofmt_xdate()
    plt.xlabel(gas)


    plt.plot(x, y)
    plt.show()

# Separando test y train

In [ ]:
rows

In [ ]:
rows = len(df)
train_df = df[0:rows-17]
test_df = df[rows-17:]
print(len(train_df), len(test_df))

## Pasos para construir modelos ARIMA
1. Identificación
2. Estimación
3. Validación
4. Predicción

In [ ]:
def make_timeline(column):
  plt.rcParams["figure.figsize"] = (20,5.5)
  mediaGasoline = train_df[column].rolling(window=12).mean()
  deGasoline = train_df[column].rolling(window=12).std()

  original = plt.plot(train_df[column], color="blue", label="Original")
  media = plt.plot(mediaGasoline, color='red', label = 'Media ' + dfToUse)
  ds = plt.plot(deGasoline, color='black', label = 'Desviación Estándar ' + dfToUse)
  plt.legend(loc = 'best')
  plt.title('Media y desviación estándar ' + column)
  plt.show(block=False)

In [ ]:
make_timeline('Gasolina regular')

In [ ]:
make_timeline('Gasolina superior')

In [ ]:
make_timeline('Diesel')

In [ ]:
train_regular = train_df[['Fecha', 'Gasolina regular']]
train_superior = train_df[['Fecha', 'Gasolina superior']]
train_diesel = train_df[['Fecha', 'Diesel']]

In [ ]:
# Gasolina regular
train_regular['Gasolina regular'] = train_regular['Gasolina regular'].astype(float)
train_regular_indexed = train_regular.set_index(['Fecha'])
# Gasolina superior
train_superior['Gasolina superior'] = train_superior['Gasolina superior'].astype(float)
train_superior_indexed = train_superior.set_index(['Fecha'])
# Gasolina diesel
train_diesel['Diesel'] = train_diesel['Diesel'].astype(float)
train_diesel_indexed = train_diesel.set_index(['Fecha'])

In [ ]:
train_regular_indexed

In [ ]:
descomposicion = seasonal_decompose(train_regular_indexed)
descomposicion.plot()

In [ ]:
descomposicion = seasonal_decompose(train_superior_indexed)
descomposicion.plot()

In [ ]:
descomposicion = seasonal_decompose(train_diesel_indexed)
descomposicion.plot()

# Estimación

In [ ]:
train_regular_indexed = train_regular_indexed[0:240].append(train_regular_indexed[248:])
train_superior_indexed = train_superior[0:240].append(train_regular_indexed[248:])
train_diesel_indexed = train_diesel[0:240].append(train_regular_indexed[248:])

In [ ]:
train_regular_indexed

Estacionar en varianza

In [ ]:
train_regular_gas = train_regular_indexed['Gasolina regular']

In [ ]:
train_regular_log = np.log(train_regular_gas)
plt.plot(train_regular_log)

Con esta transformación pudimos estacionarizarla en varianza debido a que los picos se mantienen

In [ ]:
print('Resultados del Test de Dickey Fuller')
dfTest = adfuller(train_regular_gas, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

El p-value es mayor a 0.05 por lo que no se puede rechazar la hipótesis nula de que existen raices unitarias. La serie no es estacionaria en media. Vamos a probar con una diferenciación

In [ ]:
print('Resultados del Test de Dickey Fuller para una diferenciación de la serie')
train_regular_gas_log_diff = train_regular_gas.diff()
train_regular_gas_log_diff.dropna(inplace=True)
dfTest = adfuller(train_regular_gas_log_diff)
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

El p-value es menor a 0.05 por lo que se rechaza la hipótesis nula. La serie es estacionaria en media con un d=1

In [ ]:
plt.plot(train_regular_gas_log_diff)

### Funciones de autocorrelación
#### Modelos teóricos


| Modelo    |                FAC                |                FACP               |
|-----------|:---------------------------------:|:---------------------------------:|
|   **MA(q)**   |         Se anula para j>q         | Decrecimiento rápido. No se anula |
|   **AR(p)**   | Decrecimiento rápido. No se anula |         Se anula para j>p         |
| **ARMA(p,q)** | Decrecimiento rápido. No se anula | Decrecimiento rápido. No se anula |  

In [ ]:
train_regular_gas_diff = train_regular_log.diff()
train_regular_gas_diff.dropna(inplace = True)
tsa_acf = acf(train_regular_gas_diff,nlags=5,fft=False)
tsa_pacf = pacf(train_regular_gas_diff, nlags=36)
tsa_acf

In [ ]:

plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

#Plot ACF: 
plt.subplot(121) 
plt.plot(acf(train_regular_gas_diff,nlags=36,fft=False))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación con 36 retardos')

plt.subplot(122) 
plt.plot(acf(train_regular_gas_diff,nlags=5,fft=False))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación con 5 retardos')

plt.tight_layout()

In [ ]:
#plot PACF
plt.subplot(121)
plt.plot(pacf(train_regular_gas_diff, nlags=36))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación Parcial 36 retardos')

plt.subplot(122)
plt.plot(pacf(train_regular_gas_diff, nlags=5))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación Parcial 5 retardos')

plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

- Al verificar ACF el modelo no se anula
- Al verificar PACF el modelo no se anula

---> Se acerca a un ARMA

#### Estacionalidad

In [ ]:
plt.plot(acf(train_regular_gas_diff,nlags=36,fft=False))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(train_regular_gas_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación con 36 retardos')
plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

Como se puede observar el comportamiento es similar entre 3 - 12 y 15 - 24 (estacionalidad)

In [ ]:
train_regular_log_diff = train_regular_log.diff(12)
train_regular_log_diff.dropna(inplace=True)

In [ ]:

plt.plot(pacf(train_regular_log_diff, nlags=8))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(train_regular_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(train_regular_log_diff)),linestyle='--',color='gray')
plt.title('Función de Autocorrelación Parcial 8 retardos')


Luego de hacer una diferenciación estacional, podemos observar que prácticamente se anulan los coeficientes después de p=3. 
- P = 3
- D = 1
- Q = 0

### Analisis residuos

In [ ]:
modelo121 = SARIMAX(train_regular_log, order=(1,2,1), seasonal_order=(3,1,0,12), enforce_stationarity=False, enforce_invertibility=False)
resultado_m121 = modelo121.fit()
print(resultado_m121.summary().tables[1])

In [ ]:
resultado_m121.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
modelo221 = SARIMAX(train_regular_log, order=(2,2,1), seasonal_order=(2,1,0,12), enforce_stationarity=False, enforce_invertibility=False)
resultado_m221 = modelo221.fit()
print(resultado_m221.summary().tables[1])

In [ ]:
resultado_m221.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
print("Resultados de AIC (Akaike information criterion)")
print("Modelo 121=",resultado_m121.aic)
print("Modelo 221=",resultado_m221.aic)
print("Resultados de BIC (Bayesian information criterion)")
print("Modelo 121=",resultado_m121.bic)
print("Modelo 221=",resultado_m221.bic)

De acuerdo a ambos indicadores es mejor el modelo p=2, d=2, q=1 por lo que este es el que será usado para predecir pues tinene un valor menor en AIC y en BIC